# Fórmulas Importantes no Backpropagation - Parte 2

## Introdução

Neste notebook, exploraremos as fórmulas matemáticas para o cálculo do erro nas camadas ocultas e a atualização dos pesos durante o algoritmo de Backpropagation.

In [2]:
# Importação das bibliotecas necessárias
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML, display, Math
import matplotlib.colors as mcolors
from matplotlib import cm

## Erro nas Camadas Ocultas

Enquanto o erro na camada de saída é calculado diretamente comparando a saída da rede com o valor desejado, o erro nas camadas ocultas é calculado propagando o erro da camada seguinte para trás.

### Derivação da Fórmula

Para um neurônio $j$ na camada $l$, o erro $\delta_j^l$ é definido como a derivada parcial do erro total em relação à soma ponderada $z_j^l$:

$\delta_j^l = \frac{\partial E}{\partial z_j^l}$

Usando a regra da cadeia, podemos expressar este erro em termos do erro na camada seguinte $l+1$:

$\delta_j^l = \left( \sum_{k} w_{kj}^{l+1} \delta_k^{l+1} \right) \cdot f'(z_j^l)$

Onde:
- $w_{kj}^{l+1}$ é o peso da conexão entre o neurônio $j$ na camada $l$ e o neurônio $k$ na camada $l+1$
- $\delta_k^{l+1}$ é o erro do neurônio $k$ na camada $l+1$
- $f'(z_j^l)$ é a derivada da função de ativação avaliada em $z_j^l$

In [3]:
import numpy as np

# Funções auxiliares
def sigmoid(z):
    """Função de ativação sigmoid"""
    return 1.0 / (1.0 + np.exp(-z))

def sigmoid_derivative(z):
    """Derivada da função sigmoid"""
    return sigmoid(z) * (1 - sigmoid(z))

# Cálculo do erro na camada oculta
def hidden_layer_error(delta_next, weights_next, z_current, activation_function='sigmoid'):
    """Calcula o erro para uma camada oculta"""
    
    print("\n================= INICIANDO CÁLCULO DO ERRO NA CAMADA OCULTA =================")
    
    print("\nDados de entrada:")
    print(f" Erro na camada seguinte (delta_next):\n{delta_next}")
    print(f" Pesos da camada atual para a próxima (weights_next):\n{weights_next}")
    print(f" Soma ponderada das entradas da camada atual (z_current):\n{z_current}")
    print(f" Função de ativação utilizada: {activation_function}")
    
    # 1. Calcula contribuição do erro da próxima camada
    error_contribution = np.dot(weights_next.T, delta_next)
    print("\n Contribuição do erro da camada seguinte (peso.T x delta_next):")
    print(error_contribution)
    
    # 2. Calcula a derivada da função de ativação
    if activation_function == 'sigmoid':
        activation_derivative = sigmoid_derivative(z_current)
    elif activation_function == 'tanh':
        activation_derivative = 1 - np.tanh(z_current) ** 2
    elif activation_function == 'relu':
        activation_derivative = np.where(z_current > 0, 1, 0)
    else:
        raise ValueError(f"Função de ativação '{activation_function}' não suportada")
    
    print("\n Derivada da função de ativação aplicada à camada atual:")
    print(activation_derivative)
    
    # 3. Calcula o erro da camada atual
    delta = error_contribution * activation_derivative
    
    print("\n Resultado final - Erro na camada atual (delta):")
    print(delta)
    
    print("\n================= FIM DO CÁLCULO =================\n")
    return delta

# ================= EXEMPLO DE USO =================
# Rede com 3 neurônios na camada l+1 e 2 neurônios na camada l
delta_next = np.array([[0.1], [0.2], [0.3]])  # Erro na camada seguinte (l+1)
weights_next = np.array([[0.1, 0.2],   # Pesos de cada neurônio da camada l+1 para camada l
                          [0.3, 0.4],
                          [0.5, 0.6]])
z_current = np.array([[0.5], [0.7]])  # Soma ponderada na camada atual (l)

# Cálculo do erro na camada atual (l)
delta_current = hidden_layer_error(delta_next, weights_next, z_current, activation_function='sigmoid')

print("Delta (erro) na camada atual calculado com sucesso:")
print(delta_current)



================= INICIANDO CÁLCULO DO ERRO NA CAMADA OCULTA =================

Dados de entrada:
 Erro na camada seguinte (delta_next):
[[0.1]
 [0.2]
 [0.3]]
 Pesos da camada atual para a próxima (weights_next):
[[0.1 0.2]
 [0.3 0.4]
 [0.5 0.6]]
 Soma ponderada das entradas da camada atual (z_current):
[[0.5]
 [0.7]]
 Função de ativação utilizada: sigmoid

 Contribuição do erro da camada seguinte (peso.T x delta_next):
[[0.22]
 [0.28]]

 Derivada da função de ativação aplicada à camada atual:
[[0.23500371]
 [0.22171287]]

 Resultado final - Erro na camada atual (delta):
[[0.05170082]
 [0.0620796 ]]

================= FIM DO CÁLCULO =================

Delta (erro) na camada atual calculado com sucesso:
[[0.05170082]
 [0.0620796 ]]


## Atualização dos Pesos

Após calcular os erros para cada neurônio na rede, o próximo passo é atualizar os pesos para minimizar o erro. Isso é feito usando o algoritmo do gradiente descendente ou suas variantes.

### Derivação da Fórmula

Para atualizar um peso $w_{ji}^l$ que conecta o neurônio $i$ na camada $l-1$ ao neurônio $j$ na camada $l$, a fórmula é:

$w_{ji}^l = w_{ji}^l - \eta \frac{\partial E}{\partial w_{ji}^l} = w_{ji}^l - \eta \delta_j^l a_i^{l-1}$

Onde:
- $\eta$ é a taxa de aprendizado
- $\delta_j^l$ é o erro do neurônio $j$ na camada $l$
- $a_i^{l-1}$ é a ativação do neurônio $i$ na camada $l-1$

De forma similar, a atualização dos vieses (bias) é dada por:

$b_j^l = b_j^l - \eta \delta_j^l$

In [1]:
import numpy as np

# =========================
# Implementação da atualização de pesos e vieses
# =========================

def update_weights_and_biases(weights, biases, deltas, activations, learning_rate):
    """Atualiza os pesos e vieses usando o gradiente descendente"""
    for l in range(len(weights)):
        weights[l] = weights[l] - learning_rate * np.dot(deltas[l], activations[l].transpose())
        biases[l] = biases[l] - learning_rate * deltas[l]
    return weights, biases


# =========================
# Exemplo de uso
# =========================

# Definir uma rede neural simples com 2 camadas
weights = [np.array([[0.1, 0.2], [0.3, 0.4]]), np.array([[0.5, 0.6]])]
biases = [np.array([[0.1], [0.2]]), np.array([[0.3]])]

# Definir ativações e erros (deltas)
activations = [np.array([[0.5], [0.6]]), np.array([[0.7], [0.8]]), np.array([[0.9]])]
deltas = [np.array([[0.1], [0.2]]), np.array([[0.3]])]

# Definir taxa de aprendizado
learning_rate = 0.1

# Salvar os pesos e vieses originais para comparação
original_weights = [w.copy() for w in weights]
original_biases = [b.copy() for b in biases]

# Atualizar pesos e vieses
new_weights, new_biases = update_weights_and_biases(weights, biases, deltas, activations, learning_rate)


# =========================
# Impressão detalhada dos resultados
# =========================

print("\n================= RESULTADOS DA ATUALIZAÇÃO =================\n")

# Pesos
for i in range(len(original_weights)):
    print(f" Camada {i+1} - PESOS")
    print(" Pesos originais:")
    print(original_weights[i])
    print(" Pesos atualizados:")
    print(new_weights[i])
    print(" Interpretação: Cada valor foi ajustado na direção de reduzir o erro da rede. "
          "A atualização ocorre proporcional ao gradiente (delta) e à ativação da camada anterior.\n")

# Vieses
for i in range(len(original_biases)):
    print(f" Camada {i+1} - VIESES")
    print(" Vieses originais:")
    print(original_biases[i])
    print(" Vieses atualizados:")
    print(new_biases[i])
    print(" Interpretação: Os vieses foram ajustados diretamente na proporção do delta da camada, "
          "independentemente da ativação, pois o viés não depende da entrada.\n")

print("================= FIM =================\n")



================= RESULTADOS DA ATUALIZAÇÃO =================

 Camada 1 - PESOS
 Pesos originais:
[[0.1 0.2]
 [0.3 0.4]]
 Pesos atualizados:
[[0.095 0.194]
 [0.29  0.388]]
 Interpretação: Cada valor foi ajustado na direção de reduzir o erro da rede. A atualização ocorre proporcional ao gradiente (delta) e à ativação da camada anterior.

 Camada 2 - PESOS
 Pesos originais:
[[0.5 0.6]]
 Pesos atualizados:
[[0.479 0.576]]
 Interpretação: Cada valor foi ajustado na direção de reduzir o erro da rede. A atualização ocorre proporcional ao gradiente (delta) e à ativação da camada anterior.

 Camada 1 - VIESES
 Vieses originais:
[[0.1]
 [0.2]]
 Vieses atualizados:
[[0.09]
 [0.18]]
 Interpretação: Os vieses foram ajustados diretamente na proporção do delta da camada, independentemente da ativação, pois o viés não depende da entrada.

 Camada 2 - VIESES
 Vieses originais:
[[0.3]]
 Vieses atualizados:
[[0.27]]
 Interpretação: Os vieses foram ajustados diretamente na proporção do delta da camada,

## Variantes do Gradiente Descendente

Existem várias variantes do algoritmo do gradiente descendente que podem melhorar a convergência e o desempenho:

### Gradiente Descendente com Momentum

Adiciona um termo de momentum que ajuda a acelerar a convergência e evitar mínimos locais:

$v = \gamma v - \eta \nabla E$
$w = w + v$

Onde $v$ é o vetor de velocidade e $\gamma$ é o coeficiente de momentum.

In [4]:
# Implementação do gradiente descendente com momentum
def update_with_momentum(weights, biases, deltas, activations, learning_rate, velocities, momentum=0.9):
    """Atualiza os pesos e vieses usando o gradiente descendente com momentum
    
    Args:
        weights: Lista de matrizes de pesos para cada camada
        biases: Lista de vetores de viés para cada camada
        deltas: Lista de erros para cada camada
        activations: Lista de ativações para cada camada
        learning_rate: Taxa de aprendizado
        velocities: Lista de velocidades para cada camada (pesos e vieses)
        momentum: Coeficiente de momentum
        
    Returns:
        weights: Pesos atualizados
        biases: Vieses atualizados
        velocities: Velocidades atualizadas
    """
    # Descompactar velocidades
    velocity_w, velocity_b = velocities
    
    # Atualizar pesos e vieses para cada camada
    for l in range(len(weights)):
        # Calcular gradientes
        grad_w = np.dot(deltas[l], activations[l].transpose())
        grad_b = deltas[l]
        
        # Atualizar velocidades
        velocity_w[l] = momentum * velocity_w[l] - learning_rate * grad_w
        velocity_b[l] = momentum * velocity_b[l] - learning_rate * grad_b
        
        # Atualizar pesos e vieses
        weights[l] = weights[l] + velocity_w[l]
        biases[l] = biases[l] + velocity_b[l]
    
    return weights, biases, (velocity_w, velocity_b)

# Exemplo de uso
# Inicializar velocidades com zeros
velocity_w = [np.zeros_like(w) for w in weights]
velocity_b = [np.zeros_like(b) for b in biases]
velocities = (velocity_w, velocity_b)

# Atualizar pesos e vieses com momentum
new_weights, new_biases, new_velocities = update_with_momentum(weights, biases, deltas, activations, learning_rate, velocities)


print("\n================= RESULTADOS DA ATUALIZAÇÃO COM MOMENTUM =================\n")

# Pesos
for i in range(len(new_weights)):
    print(f"Camada {i+1} - PESOS")
    print(" Pesos atualizados com momentum:")
    print(new_weights[i])
    print("Interpretação:")
    print(
        f"Nesta camada, os pesos foram atualizados considerando não apenas o gradiente atual "
        f"(que aponta na direção do erro atual), mas também a velocidade acumulada de atualizações "
        f"anteriores. O momentum (coeficiente = {0.9}) faz com que a rede tenha 'inércia' nas atualizações, "
        f"ajudando a acelerar nas direções corretas e a reduzir oscilações, especialmente em vales estreitos "
        f"do espaço de erro.\n"
    )

# Vieses
for i in range(len(new_biases)):
    print(f"Camada {i+1} - VIESES")
    print(" Vieses atualizados com momentum:")
    print(new_biases[i])
    print("Interpretação:")
    print(
        f"Os vieses foram atualizados com a mesma lógica dos pesos: além do gradiente atual, "
        f"a velocidade anterior influencia a direção e o tamanho do passo. Isso permite um "
        f"avanço mais suave e rápido na superfície de erro, evitando quedas abruptas e ajudando "
        f"na estabilidade da aprendizagem.\n"
    )

# Velocidades
for i in range(len(new_velocities[0])):
    print(f"Camada {i+1} - VELOCIDADES DOS PESOS")
    print(new_velocities[0][i])
    print("Estas velocidades representam a combinação entre a velocidade anterior "
          "e o gradiente atual, ponderadas pelo momentum. Quanto maior o momentum, "
          "mais a velocidade anterior influencia.\n")

for i in range(len(new_velocities[1])):
    print(f"Camada {i+1} - VELOCIDADES DOS VIESES")
    print(new_velocities[1][i])
    print("Velocidades aplicadas aos vieses, funcionando como um 'acúmulo' de direção "
          "para suavizar e acelerar a convergência.\n")

print("================= FIM =================\n")



================= RESULTADOS DA ATUALIZAÇÃO COM MOMENTUM =================

Camada 1 - PESOS
 Pesos atualizados com momentum:
[[0.09  0.188]
 [0.28  0.376]]
Interpretação:
Nesta camada, os pesos foram atualizados considerando não apenas o gradiente atual (que aponta na direção do erro atual), mas também a velocidade acumulada de atualizações anteriores. O momentum (coeficiente = 0.9) faz com que a rede tenha 'inércia' nas atualizações, ajudando a acelerar nas direções corretas e a reduzir oscilações, especialmente em vales estreitos do espaço de erro.

Camada 2 - PESOS
 Pesos atualizados com momentum:
[[0.458 0.552]]
Interpretação:
Nesta camada, os pesos foram atualizados considerando não apenas o gradiente atual (que aponta na direção do erro atual), mas também a velocidade acumulada de atualizações anteriores. O momentum (coeficiente = 0.9) faz com que a rede tenha 'inércia' nas atualizações, ajudando a acelerar nas direções corretas e a reduzir oscilações, especialmente em vales es

## Resumo

1. O erro em uma camada oculta é calculado como a soma ponderada dos erros na camada seguinte, multiplicada pela derivada da função de ativação: $\delta_j^l = \left( \sum_{k} w_{kj}^{l+1} \delta_k^{l+1} \right) \cdot f'(z_j^l)$.

2. A atualização dos pesos é feita usando o gradiente descendente: $w_{ji}^l = w_{ji}^l - \eta \delta_j^l a_i^{l-1}$.

3. Variantes como o gradiente descendente com momentum podem melhorar a convergência e o desempenho do algoritmo.

4. A escolha da taxa de aprendizado é crucial para o sucesso do treinamento da rede neural.